# Working With TileMatrixSets (other than WebMercator)


[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/developmentseed/titiler/main?filepath=docs%2Fexamples%2F%2Fnotebooks%2FWorking_with_nonWebMercatorTMS.ipynb)

TiTiler has builtin support for serving tiles in multiple Projections by using [rio-tiler](https://github.com/cogeotiff/rio-tiler) and [morecantile](https://github.com/developmentseed/morecantile).

This Notebook shows how to use and display tiles with non-webmercator TileMatrixSet

#### Requirements
- ipyleaflet
- httpx

In [ ]:
# Uncomment if you need to install those module within the notebook
# !pip install ipyleaflet httpx

In [ ]:
import json

import httpx

from ipyleaflet import (
    Map,
    basemaps,
    basemap_to_tiles,
    TileLayer,
    WMSLayer,
    GeoJSON,
    projections,
)

In [ ]:
titiler_endpoint = (
    "https://titiler.xyz"  # Developmentseed Demo endpoint. Please be kind.
)
url = "https://s3.amazonaws.com/opendata.remotepixel.ca/cogs/natural_earth/world.tif"  # Natural Earth WORLD tif

### List Supported TileMatrixSets

In [ ]:
r = httpx.get(f"{titiler_endpoint}/tileMatrixSets").json()

print("Supported TMS:")
for tms in r["tileMatrixSets"]:
    print("-", tms["id"])

## WebMercator - EPSG:3857

https://epsg.io/3857

In [ ]:
r = httpx.get(
    f"{titiler_endpoint}/cog/WebMercatorQuad/tilejson.json", params={"url": url}
).json()

m = Map(center=(0, 0), zoom=2, basemap={}, crs=projections.EPSG3857)

layer = TileLayer(url=r["tiles"][0], opacity=1)
m.add_layer(layer)
m

## WGS 84 -- WGS84 - World Geodetic System 1984 - EPSG:4326

https://epsg.io/4326

In [ ]:
r = httpx.get(
    f"{titiler_endpoint}/cog/WorldCRS84Quad/tilejson.json", params={"url": url}
).json()

m = Map(center=(0, 0), zoom=1, basemap={}, crs=projections.EPSG4326)

layer = TileLayer(url=r["tiles"][0], opacity=1)
m.add_layer(layer)
m

## ETRS89-extended / LAEA Europe - EPSG:3035

https://epsg.io/3035

In [ ]:
r = httpx.get(
    f"{titiler_endpoint}/cog/EuropeanETRS89_LAEAQuad/tilejson.json", params={"url": url}
).json()

my_projection = {
    "name": "EPSG:3035",
    "custom": True,  # This is important, it tells ipyleaflet that this projection is not on the predefined ones.
    "proj4def": "+proj=laea +lat_0=52 +lon_0=10 +x_0=4321000 +y_0=3210000 +ellps=GRS80 +towgs84=0,0,0,0,0,0,0 +units=m +no_defs",
    "origin": [6500000.0, 5500000.0],
    "resolutions": [8192.0, 4096.0, 2048.0, 1024.0, 512.0, 256.0],
}

m = Map(center=(50, 65), zoom=0, basemap={}, crs=my_projection)

layer = TileLayer(url=r["tiles"][0], opacity=1)
m.add_layer(layer)
m